## Classical SDP relaxation of OPF
As a first step, we present a simple implementation of the classical SDP formulation of the Optimnal Power Flow problem using CVXPY.

### Setup
We start by importing pandapower for the test cases;
CVXPY, an open source Python-embedded modeling language for convex optimization problems;
MOSEK, a large scale optimization software. 

In [1]:
import pandapower.networks as nw
import pandapower as pp
import numpy as np
import cvxpy as cp
from scipy.sparse import csr_matrix
import mosek

### 1st-Example
Let's first construct a small, 5-bus grid ourselves to really understand each step.

#### Define Variables and admittance matrix

In [2]:
n = 5  # Number of buses
G = [0, 1]  # Generator buses are either on or off
N = range(n)  # All buses
Y = np.random.randn(n, n) + 1j * np.random.randn(n, n)  # Admittance matrix
c2 = np.array([0.1, 0.2])  # Quadratic cost coefficients
c1 = np.array([10, 20])  # Linear cost coefficients
c0 = np.array([100, 200])  # Constant cost coefficients
P_min = np.array([0, 0])  # Minimum active power
P_max = np.array([100, 150])  # Maximum active power
Q_min = np.array([-50, -50])  # Minimum reactive power
Q_max = np.array([50, 50])  # Maximum reactive power
V_min = 0.95  # Minimum voltage magnitude
V_max = 1.05  # Maximum voltage magnitude
S_max = 200  # Maximum apparent power flow

#### Define Optimization Variables
Let's define the power flow constraint variables (P, Q) and the PSD matrix (X).

In [3]:
# Define optimization variables
W = cp.Variable((n, n), hermitian=True)  # X = VV^+
P = cp.Variable(len(G))  # Active power generation
Q = cp.Variable(len(G))  # Reactive power generation

#### Define Objective function

In [4]:
# Define objective
objective = cp.Minimize(cp.sum(c2 @ cp.square(P) + c1 @ P + c0))

#### Constraints

In [5]:
constraints = [W >> 0]  # X is positive semidefinite
for i in G:
    constraints += [P_min[i] <= P[i], P[i] <= P_max[i]]
    constraints += [Q_min[i] <= Q[i], Q[i] <= Q_max[i]]
for i in N:
    constraints += [V_min**2 <= cp.real(W[i, i]), cp.real(W[i, i]) <= V_max**2]
for i in N:
    for j in N:
        if i != j:
            S_ij = Y[i, j].conj() * W[i, j]
            constraints += [cp.abs(S_ij) <= S_max]

#### Solve the problem using cvxpy's SDP solvers. (probably IPM)

In [6]:
problem = cp.Problem(objective, constraints)
# Solve using a free solver (e.g., ECOS)
problem.solve()

# Print results
print("Optimal value:", problem.value)
print("X matrix:", W.value)
print("P:", P.value)
print("Q:", Q.value)

Optimal value: 300.0000000001779
X matrix: [[1.00235207+0.j 0.        +0.j 0.        +0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 1.00235207+0.j 0.        +0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 1.00235207+0.j 0.        +0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 1.00235207+0.j
  0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j
  1.00235207+0.j]]
P: [2.82054517e-12 3.03767099e-12]
Q: [-50. -50.]


/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


### Load Network Data
Now,let's make a function that loads the IEEE test cases from Pandapower.

In [2]:
# Select IEEE test case
def load_network(case):
    if case == 14:
        return nw.case14()
    elif case == 30:
        return nw.case30()
    elif case == 57:
        return nw.case57()
    elif case == 118:
        return nw.case118()
    else:
        raise ValueError("Unsupported test case")

In [3]:
#make sure all units are consistent (convert everything to per unit)
Sbase = 100 # unit value for power MVA
Vbase = 135 # unit value for voltage kV
Zbase = Vbase**2 / Sbase
Ybase = 1 / Zbase
#get the cost matrix by solving 

# Load the chosen IEEE test case
case_number = 14 # Change to 30, 57, or 118 for larger cases
net = load_network(case_number)
pp.runpp(net)  # Run power flow to initialize values
net

This pandapower network includes the following parameter tables:
   - bus (14 elements)
   - load (11 elements)
   - gen (4 elements)
   - shunt (1 element)
   - ext_grid (1 element)
   - line (15 elements)
   - trafo (5 elements)
   - poly_cost (5 elements)
   - bus_geodata (14 elements)
 and the following results tables:
   - res_bus (14 elements)
   - res_line (15 elements)
   - res_trafo (5 elements)
   - res_ext_grid (1 element)
   - res_load (11 elements)
   - res_shunt (1 element)
   - res_gen (4 elements)

In [4]:
net.bus

,name,vn_kv,type,zone,in_service,max_vm_pu,min_vm_pu
0,1,135.000,b,1.0,True,1.06,0.94
1,2,135.000,b,1.0,True,1.06,0.94
2,3,135.000,b,1.0,True,1.06,0.94
3,4,135.000,b,1.0,True,1.06,0.94
4,5,135.000,b,1.0,True,1.06,0.94
5,6,0.208,b,1.0,True,1.06,0.94
6,7,14.000,b,1.0,True,1.06,0.94
7,8,12.000,b,1.0,True,1.06,0.94
8,9,0.208,b,1.0,True,1.06,0.94
9,10,0.208,b,1.0,True,1.06,0.94


In [5]:
net.poly_cost

,element,et,cp0_eur,cp1_eur_per_mw,cp2_eur_per_mw2,cq0_eur,cq1_eur_per_mvar,cq2_eur_per_mvar2
0,0,ext_grid,0.0,20.0,0.043029,0.0,0.0,0.0
1,0,gen,0.0,20.0,0.250000,0.0,0.0,0.0
2,1,gen,0.0,40.0,0.010000,0.0,0.0,0.0
3,2,gen,0.0,40.0,0.010000,0.0,0.0,0.0
4,3,gen,0.0,40.0,0.010000,0.0,0.0,0.0


When a power flow is carried out, the element based grid model is translated into a bus-branch model. That bus-branch model is stored in a data structure that is based on the PYPOWER/MATPOWER casefile (with some extensions). This ppc can be accessed after power flow using net._ppc
We will get the bus admittance matrix using exactly that and extract all the parameters we need.

In [6]:
#Gneration bus information
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,slack_weight,type,controllable,max_p_mw,min_p_mw
0,None,1,40.0,1.045,NaN,-40.0,50.0,1.0,False,True,0.0,None,True,140.0,0.0
1,None,2,0.0,1.010,NaN,0.0,40.0,1.0,False,True,0.0,None,True,100.0,0.0
2,None,5,0.0,1.070,NaN,-6.0,24.0,1.0,False,True,0.0,None,True,100.0,0.0
3,None,7,0.0,1.090,NaN,-6.0,24.0,1.0,False,True,0.0,None,True,100.0,0.0


In [7]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type,controllable
0,None,1,21.7,12.7,0.0,0.0,NaN,1.0,True,None,False
1,None,2,94.2,19.0,0.0,0.0,NaN,1.0,True,None,False
2,None,3,47.8,-3.9,0.0,0.0,NaN,1.0,True,None,False
3,None,4,7.6,1.6,0.0,0.0,NaN,1.0,True,None,False
4,None,5,11.2,7.5,0.0,0.0,NaN,1.0,True,None,False
5,None,8,29.5,16.6,0.0,0.0,NaN,1.0,True,None,False
6,None,9,9.0,5.8,0.0,0.0,NaN,1.0,True,None,False
7,None,10,3.5,1.8,0.0,0.0,NaN,1.0,True,None,False
8,None,11,6.1,1.6,0.0,0.0,NaN,1.0,True,None,False
9,None,12,13.5,5.8,0.0,0.0,NaN,1.0,True,None,False


In [8]:
# Now, we extract network parameters
Ybus = net._ppc["internal"]["Ybus"].todense() # Bus admittance matrix (we need todense since the matrices are stored as sparcse matrices, but dense are easier to work with)
n = len(net.bus)  # Number of buses
# slack_bus = net.gen.bus.values[0]  # Index of the slack bus
branch = net.line  # Get each transmission line's specification for line constraints
#we need to add line flow constraints

# Extract generator information for generation constraints
gen_buses = net.gen["bus"].values #index of the generators.
P_min_gen = net.gen["min_p_mw"].values/net.sn_mva #minimal real power generation limit per unit
P_max_gen = net.gen["max_p_mw"].values/net.sn_mva #maximum real power generation limit per unit
Q_min_gen = net.gen["min_q_mvar"].values/net.sn_mva #minimal reactive power generation limit
Q_max_gen = net.gen["max_q_mvar"].values/net.sn_mva #maximum real power generation limit

# Extract load buses for generation constraints
load_buses = net.load["bus"].values #index of the load buses.
#load active and reactive power for load
p_load = np.zeros(n)
q_load = np.zeros(n)
for i,row in net.load.iterrows():
    p_load[row['bus']] = -row['p_mw']/net.sn_mva #negative sign because pandapower defines load as positive, but we need it as negative for consumption
    q_load[row['bus']] = -row['q_mvar']/net.sn_mva

# Voltage limits
V_min = net.bus["min_vm_pu"].values ** 2  # Squared for SDP
V_max = net.bus["max_vm_pu"].values ** 2

# Extract generator cost coefficients from net.poly_cost
cost = net.poly_cost.drop([0])
c2 = cost["cp2_eur_per_mw2"].values  # Quadratic cost coefficients
c1 = cost["cp1_eur_per_mw"].values   # Linear cost coefficients
c0 = cost["cp0_eur"].values          # Constant cost coefficients

# Ensure the cost coefficients have the correct size
if len(c2) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
if len(c1) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")
if len(c0) != len(gen_buses):
    print("The number of generators does not match the number of cost coefficients")

### Define Variable
Define the SDP Matrix X in CVXPY. We want X to be PSD. If X is rank-1, then our solution is the global optimal. In most power system cases, X is rank-1.
Represents voltage relations in SDP relaxation.

In [9]:
# Define an arbitrary n x n Hermitian W (Directly enforcing W= V V^+ will require X to be rank-1, which is too strict. We use the power balance, PSD and voltage constraints to guide this arbitrary X to X = VV^+)
W = cp.Variable((n, n), hermitian=True)

# Define phi and psi matrices for each bus as the constraints
Phi = []
Psi = []
J = []
for j in range(n):
    Y_j = np.zeros_like(Ybus,dtype=complex) #Y_j is the admittance matrix with only the j-th row, everything else is zero
    Y_j[j,:] = Ybus[j,:]
    Y_j_H = np.conj(Y_j).T #Hermitian of Y_j
    
    J_j= np.zeros_like(Ybus,dtype=complex)
    J_j[j,j] = 1

    Phi_j = 1/2 * (Y_j_H + Y_j) 
    Psi_j = 1/(2j) * (Y_j_H - Y_j)

    Phi.append(Phi_j)  # We keep them complex
    Psi.append(Psi_j)
    J.append(J_j)


# # Objective function: Minimize Pg, the matrix representing the real power generation at each generator 
# Pg = cp.real(cp.trace(W @ Ybus))  # P_g = XY. The trace sums up all the power generation and estimate the cost. We also ensure Pg is real-valued
# c2 = np.array([0.1, 0.2])  # Quadratic cost coefficients
# c1 = np.array([10, 20])  # Linear cost coefficients
# objective = cp.Minimize(cp.sum(c2 * Pg**2 + c1 * Pg)) #Quadratic Objective function

# Objective Function
objective_terms = [
                   c1[i] * cp.real(cp.trace(Ybus[int(bus), :] @ W @ Ybus[:, int(bus)])) +
                   c0[i] for i, bus in enumerate(gen_buses)]
objective = cp.Minimize(cp.sum(objective_terms))

# Constraints
constraints = []
# Positive semidefinite constraint
constraints.append(W >> 0)
# constraints.append(W[slack_bus, slack_bus] == 1)  # Slack bus voltage is 1


for j in range(n):

    #Voltage magnitude constraints
    constraints.append(V_min[j]<=cp.real(cp.trace(J[j] @ W)))
    constraints.append(cp.real(cp.trace(J[j] @ W))<=V_max[j])

    # Power balance equations (real and reactive)
    Pin_j = cp.trace(Phi[j]@W)
    Qin_j = cp.trace(Psi[j]@W)

    # If bus is a generator bus, enforce generation limits
    if j in gen_buses:
        idx = np.where(gen_buses == j)[0][0]  # Get generator index
        constraints.append(P_min_gen[idx] <= cp.real(Pin_j))
        constraints.append(cp.real(Pin_j) <= P_max_gen[idx])
        constraints.append(Q_min_gen[idx] <= cp.real(Qin_j))
        constraints.append(cp.real(Qin_j) <= Q_max_gen[idx])
    else:
        # For non-generator buses, enforce power balance constraints
        constraints.append(cp.real(Pin_j) == p_load[j])
        constraints.append(cp.real(Qin_j) == q_load[j])


In [10]:
pp.runopp(net, verbose=True)

gen vm_pu > bus max_vm_pu for gens [2 3]. Setting bus limit for these gens.


PYPOWER Version 5.1.4, 27-June-2018 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!

Converged in 0.13 seconds
Objective Function Value = 8081.53 $/hr
| PyPower (ppci) System Summary - these are not valid for pandapower DataFrames|

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             14     Total Gen Capacity     772.4         -52.0 to 148.0
Generators         5     On-line Capacity       772.4         -52.0 to 148.0
Committed Gens     5     Generation (actual)    268.3              67.6
Loads             11     Load                   259.0              73.5
  Fixed           11       Fixed                259.0              73.5
  Dispatchable     0       Dispatchable           0.0 of 0.0        0.0
Shunts             1     Shunt (inj)             -0.0              20.7
Branches          20     Losses (I^2 * Z)        

In [11]:
net.sn_mva

100

### Solve using cvxpy's SDP solver

In [12]:
# Solve the SDP
print("P_min:", P_min_gen)
print("P_max:", P_max_gen)
print("Q_min:", Q_min_gen)
print("Q_max:", Q_max_gen)

# prob = cp.Problem(objective, constraints)
prob = cp.Problem(cp.Maximize(0), constraints)
# print("Pg Limits:", Pg_min_gen, Pg_max_gen)
# print("Voltage Limits:", V_min, V_max)
# print("Power Demand:", p_load, q_load)
prob.solve(solver=cp.MOSEK, verbose=True)

# Print results
print("Optimal cost:", prob.value)
print("Optimal voltage matrix X:\n", W.value)

P_min: [0. 0. 0. 0.]
P_max: [1.4 1.  1.  1. ]
Q_min: [-0.4   0.   -0.06 -0.06]
Q_max: [0.5  0.4  0.24 0.24]
                                     CVXPY                                     
                                     v1.6.0                                    
(CVXPY) Mar 19 06:00:23 PM: Your problem has 196 variables, 260 constraints, and 0 parameters.
(CVXPY) Mar 19 06:00:23 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 19 06:00:23 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 19 06:00:23 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 19 06:00:23 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-----------------------------------------

/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18617: UserWarning: Argument sub in putvarboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putvarboundlist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18925: UserWarning: Argument subj in putclist: Incorrect array format causing data to be copied
  warnings.warn("Argument subj in putclist: Incorrect array format causing data to be copied");
/home/catz0449/VSDP/env/lib/python3.12/site-packages/mosek/__init__.py:18349: UserWarning: Argument sub in putconboundlist: Incorrect array format causing data to be copied
  warnings.warn("Argument sub in putconboundlist: Incorrect array format causing data to be copied");


In [13]:
# Check problem status
if prob.status == cp.OPTIMAL:
    print("Optimal cost:", prob.value)
    print("Optimal voltage matrix X:\n", W.value)
    print("Optimal real power demand Pd:\n", Pin_j.value)
    print("Optimal reactive power demand Qd:\n", Qin_j.value)
elif prob.status == cp.INFEASIBLE:
    print("The problem is infeasible.")
elif prob.status == cp.UNBOUNDED:
    print("The problem is unbounded.")
else:
    print("Solver status:", prob.status)

Optimal cost: 0.0
Optimal voltage matrix X:
 [[1.05600144+0.00000000e+00j 1.06276019-4.54999993e-03j
  1.0716426 -6.75398334e-03j 1.02015234+2.20744619e-02j
  1.02047621+1.57709409e-02j 1.04143721+2.16518681e-02j
  1.01570518+1.97238761e-02j 1.03009199-4.35776209e-02j
  1.00053011+5.77873484e-02j 0.99485021+5.62817283e-02j
  1.01058863+4.12160505e-02j 1.00303876+4.07186933e-02j
  1.00298453+4.38316255e-02j 0.9711343 +6.97930063e-02j]
 [1.06276019+4.54999993e-03j 1.07522611+0.00000000e+00j
  1.08234994-2.25061171e-03j 1.0309071 +2.68070019e-02j
  1.03085863+2.04373998e-02j 1.05292902+2.67408072e-02j
  1.02693374+2.45141374e-02j 1.04110088-3.95538140e-02j
  1.01179722+6.30428160e-02j 1.00567308+6.14306642e-02j
  1.02135771+4.62875902e-02j 1.01361806+4.56506679e-02j
  1.01405508+4.90250856e-02j 0.98175784+7.50580505e-02j]
 [1.0716426 +6.75398334e-03j 1.08234994+2.25061171e-03j
  1.10364761+0.00000000e+00j 1.04120294+2.92113772e-02j
  1.03987729+2.27285997e-02j 1.0620488 +2.89060991e-02j
 